In [7]:
# ====================================================================
# 0. Import Libraries and Load Data
# ====================================================================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import os
import pickle

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

warnings.filterwarnings("ignore")

## For Dynamic Pathing ###
from pathlib import Path
current_dir = Path.cwd()
base_dir = current_dir.parent
data_path = base_dir / 'data' / 'raw' / 'water_quality.csv'

# Load the dataset
try:
    df = pd.read_csv(data_path)
    print("✅ Dataset loaded successfully.")
except FileNotFoundError:
    print("❌ Error: 'data/raw/water_quality.csv' not found.")
    df = pd.DataFrame()

## Phase 3: Optimization & Serialization
---
In this phase, we will take the best-performing model from our baseline evaluation, **Gradient Boosting**, and optimize it using hyperparameter tuning. We will then save the final, tuned model for future use.


In [8]:
# ====================================================================
# 1. Data Preparation (Repeating the steps from Phase 2)
# ====================================================================
# This section repeats the exact data preparation steps to ensure consistency.

if not df.empty:
    # 1.1. Feature and Target Selection
    features = ['EC', 'Cl', 'TDS', 'Na']
    target = 'Water Quality Classification'
    X = df[features]
    y = df[target]

    # 1.2. Encode Target Variable
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    class_names = le.classes_

    # 1.3. Stratified Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded,
        test_size=0.2,
        random_state=42,
        stratify=y_encoded
    )

    # 1.4. Feature Scaling
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print("✅ Data preparation complete.")
    print("Training data shape:", X_train_scaled.shape)

## 2. Hyperparameter Tuning with RandomizedSearchCV
---
We use `RandomizedSearchCV` because it's much more efficient than searching every possible combination (`GridSearchCV`). It samples a fixed number of parameter settings from the specified distributions.

We will optimize for the **Macro-F1 score**, as it's a robust metric for imbalanced classification problems.


In [9]:
# ====================================================================
# 2.1. Define the Hyperparameter Grid
# ====================================================================

# Define the grid of hyperparameters to search
param_distributions = {
    "n_estimators": [100, 200, 300, 500],
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "max_depth": [3, 5, 7, 10],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "max_features": ['sqrt', 'log2']
}

# ====================================================================
# 2.2. Set up and Run RandomizedSearchCV
# ====================================================================
if not df.empty:
    # Initialize the base model
    gb_model = GradientBoostingClassifier(random_state=42)

    # Set up RandomizedSearchCV
    # n_iter=50 means it will try 50 different combinations of parameters.
    # cv=5 means 5-fold cross-validation.
    # n_jobs=-1 uses all available CPU cores to speed up the process.
    random_search = RandomizedSearchCV(
        estimator=gb_model,
        param_distributions=param_distributions,
        n_iter=50,
        cv=5,
        scoring='f1_macro',
        n_jobs=-1,
        random_state=42,
        verbose=2 # Shows progress
    )

    print("🚀 Starting hyperparameter tuning...")
    # Fit the random search to the data
    random_search.fit(X_train_scaled, y_train)
    print("✅ Tuning complete.")

🚀 Starting hyperparameter tuning...
Fitting 5 folds for each of 50 candidates, totalling 250 fits
✅ Tuning complete.


In [10]:
# ====================================================================
# 3. Analyze Tuning Results
# ====================================================================
if 'random_search' in locals():
    print("\n🏆 Best Hyperparameters Found:")
    print(random_search.best_params_)

    print(f"\n⭐ Best Macro-F1 Score from Cross-Validation: {random_search.best_score_:.4f}")
    
    # The best model found by the search
    best_model = random_search.best_estimator_

## 4. Final Evaluation of the Optimized Model
---
Now we evaluate the `best_model` (the one with the optimal hyperparameters) on the held-out test set to see how much it improved over the baseline.


In [11]:
# ====================================================================
# 4.1. Evaluate the Tuned Model on the Test Set
# ====================================================================
if 'best_model' in locals():
    y_pred_tuned = best_model.predict(X_test_scaled)

    # Evaluate the tuned model
    accuracy_tuned = accuracy_score(y_test, y_pred_tuned)
    report_tuned = classification_report(y_test, y_pred_tuned, target_names=class_names)

    print("✅ Results for Tuned Gradient Boosting Model:")
    print(f"Accuracy: {accuracy_tuned:.4f}")
    print("Classification Report:")
    print(report_tuned)

    # Visualize the final Confusion Matrix
    cm_tuned = confusion_matrix(y_test, y_pred_tuned)
    disp_tuned = ConfusionMatrixDisplay(confusion_matrix=cm_tuned, display_labels=class_names)

    fig, ax = plt.subplots(figsize=(8, 6))
    disp_tuned.plot(ax=ax, cmap='Greens', xticks_rotation='vertical')
    ax.set_title('Confusion Matrix for Tuned Gradient Boosting Model')
    
    # Save the figure
    cm_path_tuned = 'reports/figures/confusion_matrix_Tuned_Gradient_Boosting.png'
    plt.savefig(cm_path_tuned, bbox_inches='tight')
    print(f"\n✅ Final confusion matrix saved to {cm_path_tuned}")
    
    plt.show()

## 5. Model Serialization
---
The final step is to save our work. We serialize and save two crucial components:
1.  **The Tuned Model**: The `best_model` object itself, ready for making future predictions.
2.  **The Scaler**: The `MinMaxScaler` object. This is **critical** because any new data must be scaled in exactly the same way as the training data before being fed to the model.


In [12]:
# ====================================================================
# 5.1. Save the Model and Scaler to Disk
# ====================================================================
if 'best_model' in locals() and 'scaler' in locals():
    # Create the models directory if it doesn't exist
    if not os.path.exists('models'):
        os.makedirs('models')

    # Define file paths
    model_path = 'models/final_gradient_boosting_model.pkl'
    scaler_path = 'models/min_max_scaler.pkl'

    # Save the model
    with open(model_path, 'wb') as f:
        pickle.dump(best_model, f)
    print(f"💾 Final model saved to: {model_path}")

    # Save the scaler
    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)
    print(f"💾 Scaler saved to: {scaler_path}")

💾 Final model saved to: models/final_gradient_boosting_model.pkl
💾 Scaler saved to: models/min_max_scaler.pkl
